In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pymysql
import sys

from detection import generate_download_signed_url_v4
from detection import get_similar_products_uri
from detection import query_product

from tensorflow.keras import backend
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from google.cloud import storage
from google.cloud import vision
from urllib.parse import urlparse
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

In [46]:
#Setup Connection to mysql database
conn = pymysql.connect(
    host='35.221.181.94',
    port=int(3306),
    user="mkhoa",
    passwd='NTMK261194@dng',
    db="project",
    charset='utf8mb4')

cur = conn.cursor()

def query_item2room():
    '''
    Make dataframe for ImageDataGenerator
    
    '''
    query = f'''
    SELECT a.room, a.room_path, a.item, b.bucket, b.bucket_path
    FROM project.item2room a
    LEFT JOIN project.Files b ON a.item = b.id
    WHERE b.bucket_path like '%Products%'
    '''
    try:
        cur.execute(query)
    except Exception as err:
        print('ERROR BY SELECT:', err)
    result = cur.fetchall()
    result = pd.DataFrame(result, columns=['RoomID', 'RoomPath', 'ItemID', 'Bucket', 'BucketPath'])
    return result

def query_classes():
    '''
    Query for list of distinct room
    
    '''
    query = f'''
    SELECT distinct room
    FROM project.item2room
    '''
    try:
        cur.execute(query)
    except Exception as err:
        print('ERROR BY SELECT:', err)
    result = cur.fetchall()
    result = pd.DataFrame(result, columns=['RoomID'])
    return result

In [47]:
item2room = query_item2room()

In [48]:
classes = query_classes()

In [49]:
df = item2room[['RoomID', 'BucketPath']]
df_train, df_test = train_test_split(df, test_size=0.1)

In [50]:
df_train

,RoomID,BucketPath
155,30707,Images/Products/204169-0-102.335.32.jpg
138,30571,Images/Products/203973-0-603.347.36.jpg
132,30615,Images/Products/203847-0-902.857.82.jpg
65,30483,Images/Products/203346-0-902.535.35.jpg
122,30684,Images/Products/203742-0-002.387.14.jpg
...,...,...
45,30634,Images/Products/203261-0-002.110.88.jpg
27,30702,Images/Products/203046-0-702.068.04.jpg
133,30492,Images/Products/203847-0-902.857.82.jpg
158,30664,Images/Products/204221-0-702.858.58.jpg


In [51]:
project_id = 'abstract-veld-289612'
bucket_name = 'ftmle'
storage_client = storage.Client.from_service_account_json("./Credentials/abstract-veld-289612-327ddac80eba.json")
# tf.compat.v1.enable_eager_execution()

In [52]:
# Fit multi-label binarizer on the training set
lb = LabelBinarizer()
lb.fit(df_train['RoomID'])

# Print out label map 
#print('Labels: ')
label_map = {}
for i, label in enumerate(lb.classes_):
    label_map[i] = label
    #print('{}. {}'.format(i, label))

# Define num. of labels 
N_LABELS = len(lb.classes_)

# Fit multi-label binarizer on the training set
y_train = lb.transform(df_train['RoomID'])
y_val = lb.transform(df_test['RoomID'])

In [59]:
X_train = df_train['BucketPath'].values
X_test = df_test['BucketPath'].values

In [60]:
Y_train =  y_train
Y_test =  y_val

In [61]:
def load_bucket_image(path):
    path = str(path.numpy().decode("utf-8"))
    blob = storage_client.bucket(bucket_name).get_blob(path)
    img = blob.download_as_string()
    return img

def preprocess_image(bucket_path):
    '''
    Load and preprocess image from bucket path
    
    '''
    img = tf.py_function(load_bucket_image, [bucket_path], tf.string) 
    img = tf.image.decode_image(img, channels=3)
 
    return img

# The tuples are unpacked into the positional arguments of the mapped function

def load_and_preprocess_from_path_label(path, label):
  return preprocess_image(path), label

In [62]:
ds_train = tf.data.Dataset.from_tensor_slices((X_train, Y_train))
ds_train_map = ds_train.map(load_and_preprocess_from_path_label)

In [63]:
ds_test = tf.data.Dataset.from_tensor_slices((X_test, Y_test))
ds_test_map = ds_test.map(load_and_preprocess_from_path_label)